In [ ]:
from PIL import Image
import numpy as np


blocks getting function

In [ ]:
def getblocks(image: np.ndarray, blockshape: tuple, moveAxis: bool = True, info: bool = False, addChannel: bool = True) -> np.ndarray:
    '''
    takes the array of image in grey= 2D and in RGB = 3D
    takes the numpy array and converts it the the blocks in the fastest way
    '''
    if(info):
        print("Image Shape:", image.shape)
        print("Block Shape:", blockshape)

    oldshape = list(image.shape)
    if addChannel and len(image.shape) == 2:
        mode = "grey"
        image = image.reshape((*image.shape, 1))
    else:
        mode = "color"

    if addChannel:
        img_height, img_width, channels = image.shape
    else:
        img_height, img_width = image.shape

    tile_height, tile_width = blockshape

    if addChannel:
        shp = img_height//tile_height, tile_height, img_width//tile_width, tile_width, channels
    else:
        shp = img_height//tile_height, tile_height, img_width//tile_width, tile_width

    def printinfo():
        print("Old Shape:", oldshape)
        print("Image Shape:", image.shape)
        print("Block Shape:", blockshape)
        print("New Shape Initial:", shp)
        print("img_height % tile_height != 0 :", img_height % tile_height != 0)
        print("img_width % tile_width != 0 :", img_width % tile_width != 0)

    if img_height % tile_height != 0 or img_width % tile_width != 0:
        print("warning: Block size is not fit for the image!")
        printinfo()

    if(info):
        printinfo()

    tiled_array = image.reshape(shp)
    tiled_array = tiled_array.swapaxes(1, 2)

    if moveAxis:
        if(addChannel):
            tiled_array = tiled_array.reshape(-1,
                                              *(tile_height, tile_width, channels))
            tiled_array = np.moveaxis(tiled_array, source=len(
                tiled_array.shape)-1, destination=1)
        else:
            tiled_array = tiled_array.reshape(-1, *(tile_height, tile_width))

    return tiled_array


In [ ]:


def combineBlocks(tiled_array: np.ndarray, imageshape: tuple, blockshape: tuple, movedAxis: bool = True) -> np.ndarray:

    if len(imageshape) == 2:
        mode = "grey"
        imageshape = *imageshape, 1
    else:
        mode = "color"

    img_height, img_width, channels = imageshape
    tile_height, tile_width = blockshape

    if movedAxis:
        image = tiled_array.copy()
        image = image.reshape(img_height//tile_height, tile_height,
                              img_width//tile_width, tile_width, channels)
        swapaxisShape = list(image.shape)
        swapaxisShape[1], swapaxisShape[2] = swapaxisShape[2], swapaxisShape[1]
        image = image.reshape(swapaxisShape)
        image = image.swapaxes(1, 2)
    else:
        image = tiled_array
        # IDK have completed this else case. Btw we aren't using this case lol :)

    return image.reshape(imageshape)


In [ ]:
def printnd(arr: np.ndarray):
    print("Shape:", arr.shape)
    print("Strides:", arr.strides)
    print(arr)


importing image
e.g lena which is 512x512
importing as the grey image denoted by L flag

In [ ]:
lena = Image.open("fyp_image.jpg").convert("RGB")
lena


In [ ]:
lena.mode


converting to np array

In [ ]:
img = np.array(lena)
imgSize = img.shape
img.shape, img.strides, img


making 16 main blocks
size of single main block S= M/sqrt(16) X T=N/sqrt(16)
e.g for lena the 512x512 the partner blocks size would be 16 blocks each with size (512x512)/(4x4) = 128x128

In [ ]:
mainblockSize = (int(imgSize[0]/4), int(imgSize[1]/4))
mainblocks = getblocks(img.copy(), mainblockSize, info=True)
mainblocks.shape, mainblocks.strides, mainblocks


here the main blocks are like this index.
1st index is the block number.
2nd index is the channel (RGB) or 0 in Grey.
3rd and 4th are for indexing the block.

lookup table

In [ ]:
#Image.open("lookup.png")

In [ ]:
lookup = np.array([[0, 7, 13, 10],
                  [1, 6, 12, 11],
                  [4, 2, 9, 15],
                  [5, 3, 8, 14]], dtype=np.uint8)
lookup

here the every row represents the partner blocks
e.g
A1 will be lookup[0][0].
A2 will be lookup[0][2].
D2 will be lookup[3][2].


Step3
Ab har main block ko divide karo k 4x4 k block ban jain
numberOfBlocks=(SxT)/(4x4)
e.g 128x128/4x4 = 1024


In [ ]:
avgblocksSize = (4, 4)
averages = np.zeros((16, int((mainblockSize[0]*mainblockSize[1]) /
                             (avgblocksSize[0]*avgblocksSize[1]))), dtype=np.uint8)
# 4 indicatior A,B,C,D, 4 blocks of A, then the 4x4 Blocks which have count = (SxT)/(4x4), e.g 1024


storing the averages of A, B, C, D

In [ ]:
tmp=getblocks(mainblocks[0][0], avgblocksSize,
          info=False, addChannel=False)
tmp.shape, tmp

In [ ]:
lookup

In [ ]:
for partner in lookup:  # A,B,C,D
    for id in partner:  # A1,A2,A3.....D4 etc
        averages[id] = getblocks(mainblocks[id][0], avgblocksSize,
                                 info=False, addChannel=False).mean((1, 2))
averages.shape, averages


array([[162, 162, 162, 161],
       [162, 162, 162, 161],
       [162, 162, 162, 161],
       [162, 162, 162, 161]], dtype=uint8)
this will give you the avg 161. But it sould be 162 as it's dominant. Minor improvemnt reuqired

now we have average of every mainblock according to 4x4. Which in total are 1024

time to convert them into the binary

In [ ]:
averageBits = np.unpackbits(averages, axis=1)
averageBits.shape, averageBits


merging partner blocks average to make the recovery bits


In [ ]:
recoveryBits = np.zeros((averageBits.shape[0], averageBits.shape[1]*3), dtype=np.uint8)
for partner in lookup:  # A,B,C,D
    for id in partner:  # A1,A2,A3.....D4 etc
        recoveryBits[id] = np.concatenate(
            [averageBits[i] for i in partner if i != id])

recoveryBits.shape, recoveryBits


password or key generation

In [ ]:
import hashlib

In [ ]:
# password or key
password = "this is my password"
key = [x for x in hashlib.sha256(password.encode()).digest()]
len(key)


permuttaion or shuffling

In [ ]:
for i in range(recoveryBits.shape[0]):
    np.random.RandomState(seed=key).shuffle(recoveryBits[i])

recoveryBits.shape, recoveryBits


Recovery bits creation completed
Now we need to make the space to put these recovery bits

### Making 16x16 for Step 8
Dividing the main blocks to 16x16 blocks
Total blocks = SxT/16x16 = 128x128/16x16 = 64 Blocks

In [ ]:
mainblocks.shape

In [ ]:
blocks16x16Size = (16, 16)
# reshaping because we needed that shape
blocks16x16 = mainblocks.reshape(
    *mainblocks.shape[:-2],
    int((mainblockSize[0]*mainblockSize[1]) /
        (blocks16x16Size[0]*blocks16x16Size[1])),
    blocks16x16Size[0], blocks16x16Size[1])

blocks16x16Size, blocks16x16.shape, blocks16x16.strides, blocks16x16


In [ ]:
for block, mainblock in zip(blocks16x16, mainblocks):
    block[0] = getblocks(mainblock[0], blocks16x16Size,
                         info=False, addChannel=False)

blocks16x16.shape, blocks16x16.strides, blocks16x16


### Making 8x8 blocks of those 16x16  Step 9

In [ ]:
blocks16x16.shape

In [ ]:
blocks8x8Size = (8, 8)
# reshaping because we needed that shape
blocks8x8 = blocks16x16.reshape(
    *blocks16x16.shape[:-2],
    int((blocks16x16Size[0]*blocks16x16Size[1]) /
        (blocks8x8Size[0]*blocks8x8Size[1])),
    blocks8x8Size[0], blocks8x8Size[1])

blocks8x8Size, blocks8x8.shape, blocks8x8.strides, blocks8x8


In [ ]:
blocks8x8.shape

In [ ]:
for block, block16x16 in zip(blocks8x8, blocks16x16):
    for i in range(block16x16[0].shape[0]):
        block[0][i] = getblocks(block16x16[0][i], blocks8x8Size,
                                info=False, addChannel=False)

blocks8x8.shape, blocks8x8.strides, blocks8x8


Now put the recovery bit in the first and second LSB of blocks4x4


In [ ]:
blocks8x8.shape

In [ ]:
recoveryBits.reshape(recoveryBits.shape[0], int(recoveryBits.shape[1]//2), 2).shape


In [ ]:
recoveryBits[0]

In [ ]:
recoveryBits.shape, blocks8x8.shape[2]*3*8*8*2

In [ ]:
blocks8x8[:, 0, :, :3, :, :].shape, blocks8x8[:, 0, :, :3, :,
                                              :][0].shape, blocks8x8[:, 0, :, :3, :, :][0].flatten().shape


updating the lsb

In [ ]:
def set_bit(value, index, x):
    """Set the index:th bit of v to 1 if x is truthy, else to 0, and return the new value."""
    mask = 1 << index   # Compute mask, an integer with just bit 'index' set.
    # Clear the bit indicated by the mask (if x is False)
    value &= ~mask
    if x:
        # If x was True, set the bit indicated by the mask.
        value |= mask
    return value            # Return the result, we're done.


In [ ]:
set_bit(2,1,0),set_bit(2,2,1)

In [ ]:
for mainblock, blockbits in zip(blocks8x8[:, 0, :, :3, :, :], recoveryBits.reshape(recoveryBits.shape[0], int(recoveryBits.shape[1]//2), 2)):
    for number, bits in zip(tmp.flat, blockbits):
        # print(number, bits)
        number = set_bit(set_bit(number, 0, bits[0]), 1, bits[1])

blocks8x8.shape, blocks8x8.strides, blocks8x8


### Now to combine the blocks back to Image

In [ ]:
raise Exception("Below code is not in running form")

In [ ]:
avgblocksSize = (4, 4)
# reshaping because we needed that shape
avgblocks = mainblocks.reshape(
    *mainblocks.shape[:-2],
    int((mainblockSize[0]*mainblockSize[1]) /
        (avgblocksSize[0]*avgblocksSize[1])),
    avgblocksSize[0], avgblocksSize[1])

avgblocksSize, avgblocks.shape, avgblocks.strides, avgblocks


In [ ]:
block=avgblocks[0][0]
block.shape, block.strides, block

In [ ]:
avgblocks.shape, avgblocks.strides

In [ ]:
avgblocksSize

In [ ]:
for block, mainblock in zip(avgblocks, mainblocks):
    block[0] = getblocks(mainblock[0], avgblocksSize,
                         info=False, addChannel=False)

avgblocks.shape, avgblocks.strides, avgblocks


In [ ]:
avgblocks[0][0].reshape(-1).shape

In [ ]:
mainblock[0][0].reshape(-1).shape

In [ ]:
avgblocks[0][0].reshape(-1)==mainblock[0][0].reshape(-1)